#Fetch Youtube Comment for Wuling's Channel

I need to make:

1.  Get channel Id
2.  Get a list of videos id
3. For each video load comment threads
4. For each comment thread load the comment
5. for each comment thread with replies load the replies


In [1]:
from googleapiclient.discovery import build
import pandas as pd
from google.colab import files, drive
import getpass

In [2]:
api_key = '' #myAPI
channel_id = '' #ChannelID

In [3]:
# Build the YouTube client
youtube = build('youtube', 'v3', developerKey=api_key)

In [4]:
def get_all_video_ids_from_channel(youtube, channel_id):
    all_videos = []  # Initialize a single list to hold all video IDs
    next_page_token = None

    # Fetch videos from the specified channel
    while True:
        channel_request = youtube.search().list(
            part='id',
            channelId=channel_id,
            type='video',
            maxResults=50,
            pageToken=next_page_token)
        channel_response = channel_request.execute()

        all_videos += [item['id']['videoId'] for item in channel_response['items']]

        next_page_token = channel_response.get('nextPageToken')

        if next_page_token is None:
            break

    return all_videos

# Fetch all video IDs from the specified channel
video_ids = get_all_video_ids_from_channel(youtube, channel_id)

# Now you can pass video_ids to the next function
# next_function(video_ids)


In [5]:
# Function to get replies for a specific comment
def get_replies(youtube, parent_id, video_id):  # Added video_id as an argument
    replies = []
    next_page_token = None

    while True:
        reply_request = youtube.comments().list(
            part="snippet",
            parentId=parent_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        reply_response = reply_request.execute()

        for item in reply_response['items']:
            comment = item['snippet']
            replies.append({
                'Timestamp': comment['publishedAt'],
                'Username': comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': comment['textDisplay'],
                'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
            })

        next_page_token = reply_response.get('nextPageToken')
        if not next_page_token:
            break

    return replies

# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None

    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()

        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,  # Directly using video_id from function parameter
                'Comment': top_comment['textDisplay'],
                'Date': top_comment['updatedAt'] if 'updatedAt' in top_comment else top_comment['publishedAt']
            })

            # Fetch replies if there are any
            if item['snippet']['totalReplyCount'] > 0:
                all_comments.extend(get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id))

        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return all_comments

# List to hold all comments from all videos
all_comments = []


for video_id in video_ids:
    video_comments = get_comments_for_video(youtube, video_id)
    all_comments.extend(video_comments)

# Create DataFrame
comments_df = pd.DataFrame(all_comments)

In [6]:
comments_df

,Timestamp,Username,VideoID,Comment,Date
0,2019-07-23T03:48:12Z,@siburju,m4xry9lHnc4,"Review Dikit yah, ini pendapat pribadi, mohon ...",2019-07-23T03:48:12Z
1,2019-07-23T04:12:11Z,@WulingMotorsID,m4xry9lHnc4,"Selamat siang, Bapak Siburju. Terima kasih ata...",2019-07-23T04:12:11Z
2,2019-07-17T12:06:38Z,@vargassamator6650,m4xry9lHnc4,Hasil iklan nya bagus juga,2019-07-17T12:06:38Z
3,2019-07-17T08:11:36Z,@rindikennopolanda1053,m4xry9lHnc4,Untuk yang di tangerang sekitarnya \nJika ada ...,2019-07-17T08:11:36Z
4,2019-07-17T06:24:22Z,@aswaeyes1708,m4xry9lHnc4,Wuling tolong dong emblem di perkecil dan jadi...,2019-07-17T06:24:35Z
...,...,...,...,...,...
1032,2023-03-03T14:13:15Z,@cahya9388,GdHHjsk61EU,Sekedar masukan menurut saya nama wuling kuran...,2023-03-03T14:13:15Z
1033,2023-03-20T13:28:54Z,@andiharamain4526,GdHHjsk61EU,Kata siapa 😂😂😂?? klo denger suara Halo Wuling ...,2023-03-20T13:28:54Z
1034,2023-03-01T08:30:21Z,@budiamin2898,GdHHjsk61EU,"MAJU TERUS WULING, SUKSES BUAT WULING",2023-03-01T08:30:21Z
1035,2023-03-20T13:24:31Z,@andiharamain4526,GdHHjsk61EU,Semoga semakin sukses jg yg pake Wuling Alvez 🥳🥳🥳,2023-03-20T13:24:31Z


In [7]:
# Export whole dataset to the local machine as CSV File
csv_file = 'comments_dataYT.csv'  # Name your file
comments_df.to_csv(csv_file, index=False)

from google.colab import files

# Trigger a download to your local machine
files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#YouTube metrics for all contents in Wuling's Channel

In [8]:
# Retrieve channel information
# Replace 'YOUR_CHANNEL_ID' with your own channel ID
channel_response = youtube.channels().list(
    part="snippet,statistics",
    id="" #ChannelID Wuling
).execute()

# Extract channel data
channel_data = channel_response["items"][0]
channel_snippet = channel_data["snippet"]
channel_statistics = channel_data["statistics"]

# Print channel information
print("Channel Title:", channel_snippet["title"], '\n')
print("Channel Description:", channel_snippet["description"], '\n')
print("Channel Views:", channel_statistics["viewCount"])
print("Channel Subscribers:", channel_statistics["subscriberCount"])
print("Channel Videos:", channel_statistics["videoCount"])

Channel Title: Wuling MotorsID 

Channel Description: Selamat datang di kanal YouTube resmi milik Wuling Motors Indonesia. Melalui semangat "Wuling Motors - Drive For A Better Life", kami menyajikan berbagai konten video informatif untuk kenyamanan berkendara para pelanggan kami. Seperti video detail varian produk mobil Wuling, testimoni pengguna Wuling, pelayanan kepada pelanggan, hingga kegiatan CSR dan event.

Pastikan Anda subscribe kanal YouTube Wuling Motors Indonesia! 

Call Center - 0800-100-5050 

Channel Views: 72652710
Channel Subscribers: 45000
Channel Videos: 612


In [9]:
def get_all_video_ids_from_channel(youtube, channel_id):
    all_videos = []  # Initialize a single list to hold all video IDs
    next_page_token = None

    # Fetch videos from the specified channel
    while True:
        channel_request = youtube.search().list(
            part='id',
            channelId=channel_id,
            type='video',
            maxResults=50,
            pageToken=next_page_token)
        channel_response = channel_request.execute()

        all_videos += [item['id']['videoId'] for item in channel_response['items']]

        next_page_token = channel_response.get('nextPageToken')

        if next_page_token is None:
            break

    return all_videos

# Fetch all video IDs from the specified channel
video_ids = get_all_video_ids_from_channel(youtube, channel_id)

# Now you can pass video_ids to the next function
# next_function(video_ids)


In [10]:
def get_video_details(youtube, video_ids):
    video_details = []
    for video_id in video_ids:
        video_response = youtube.videos().list(
            part='snippet,statistics',
            id=video_id
        ).execute()

        if 'items' in video_response:
            video = video_response['items'][0]
            snippet = video['snippet']
            statistics = video['statistics']

            video_detail = {
                'Date': snippet['publishedAt'],
                'Video ID': video_id,
                'Title': snippet['title'],
                'Views': statistics.get('viewCount', 'N/A'),
                'Likes': statistics.get('likeCount', 'N/A'),
                'Dislikes': statistics.get('dislikeCount', 'N/A'),
                'Comments': statistics.get('commentCount', 'N/A'),
                'Shares': statistics.get('shareCount', 'N/A'),
                'Engagements': sum(int(statistics.get(key, 0)) for key in ['likeCount', 'dislikeCount', 'commentCount', 'shareCount'])
            }
            video_details.append(video_detail)

    return video_details


In [11]:
videos = get_video_details(youtube, video_ids)
videos

[{'Date': '2019-07-16T07:48:16Z',
  'Video ID': 'm4xry9lHnc4',
  'Title': 'Cortez CT - The New Advanced MPV',
  'Views': '2122582',
  'Likes': '119',
  'Dislikes': 'N/A',
  'Comments': '28',
  'Shares': 'N/A',
  'Engagements': 147},
 {'Date': '2024-02-28T11:38:25Z',
  'Video ID': 'xdpfWyzhpWM',
  'Title': 'Wuling Cloud EV: Comfort, Smooth and Elegant',
  'Views': '24425',
  'Likes': '67',
  'Dislikes': 'N/A',
  'Comments': '32',
  'Shares': 'N/A',
  'Engagements': 99},
 {'Date': '2023-04-02T11:47:31Z',
  'Video ID': 'Wbzdp7X4-iM',
  'Title': 'Yang ditunggu datang juga, PPN Air ev dari 11% jadi hanya 1%! 📢',
  'Views': '492',
  'Likes': '14',
  'Dislikes': 'N/A',
  'Comments': '1',
  'Shares': 'N/A',
  'Engagements': 15},
 {'Date': '2018-09-16T06:02:01Z',
  'Video ID': '5esZnyND8Xk',
  'Title': 'Wuling Motors - GIIAS Surabaya Auto Show 2018 #WulingGIIASSurabaya2018',
  'Views': '11236',
  'Likes': '31',
  'Dislikes': 'N/A',
  'Comments': '12',
  'Shares': 'N/A',
  'Engagements': 43},
 {

In [12]:
video_dataYT = pd.DataFrame(videos)

In [13]:
video_dataYT

,Date,Video ID,Title,Views,Likes,Dislikes,Comments,Shares,Engagements
0,2019-07-16T07:48:16Z,m4xry9lHnc4,Cortez CT - The New Advanced MPV,2122582,119,N/A,28,N/A,147
1,2024-02-28T11:38:25Z,xdpfWyzhpWM,"Wuling Cloud EV: Comfort, Smooth and Elegant",24425,67,N/A,32,N/A,99
2,2023-04-02T11:47:31Z,Wbzdp7X4-iM,"Yang ditunggu datang juga, PPN Air ev dari 11%...",492,14,N/A,1,N/A,15
3,2018-09-16T06:02:01Z,5esZnyND8Xk,Wuling Motors - GIIAS Surabaya Auto Show 2018 ...,11236,31,N/A,12,N/A,43
4,2022-01-14T12:00:40Z,xQe0LMJ6HBM,Panoramic Sunroof Almaz RS yang Bikin Betah di...,34102,591,N/A,21,N/A,612
...,...,...,...,...,...,...,...,...,...
225,2020-11-24T05:43:59Z,oEd7C9foqNU,"Wuling Almaz Limited Edition, Menawarkan Fitur...",9620,218,N/A,99,N/A,317
226,2021-07-11T04:00:13Z,PYznpQ3_AKY,Perjalanan 4 Tahun Wuling untuk Keluarga Indon...,574109,184,N/A,18,N/A,202
227,2023-05-12T03:49:22Z,UZg1qadGMcc,Inilah kesiapan Air ev menyukseskan KTT ASEAN ...,2128,30,N/A,3,N/A,33
228,2021-06-26T12:30:17Z,irjVwXVCFf4,Internet of Vehicle (IoV) Almaz RS - Cara Memb...,836,12,N/A,1,N/A,13
